# **SPAM vs. naiwny klasyfikator Bayesa**
Autor: Anna Dawid

Rozwiązanie: Zofia Hendrysiak

## Wprowadzenie
Nigeryjski książę wciąż zarabia na użytkownikach elektronicznych skrzynek pocztowych ponad 700 tys. dolarów rocznie ([źródło](https://www.cnbc.com/2019/04/18/nigerian-prince-scams-still-rake-in-over-700000-dollars-a-year.html))! Jak to możliwe?

Pierwsza przyczyna jest natury psychologicznej. Ofiary są poddawane "perfekcyjnej burzy pokuszeń", jak ujął to psycholog w wywiadzie, do którego linka dałam Wam powyżej. Spammerzy łączą granie na ludzkiej chciwości, ale także na pragnieniu bycia bohaterem. W końcu kto nie chciałby zarobić na byciu wspaniałomyślnym i szczodrym? W tej kwestii możemy pracować wyłącznie nad sobą.

Możemy za to pracować nad filtrami antyspamowymi. Użyjemy techniki, która nazywa się "worek ze słowami" (bag of words) w połączeniu z naiwnym klasyfikatorem Bayesa. Choć to prosty klasyfikator, z powodzeniem jest używany współcześnie (np. [SpamAssassin](https://cwiki.apache.org/confluence/display/spamassassin/BayesInSpamAssassin)).

Notebook oparty na tutorialach:
*   https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73
*   https://towardsdatascience.com/spam-filtering-using-naive-bayes-98a341224038


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Import danych treningowych
https://www.kaggle.com/uciml/sms-spam-collection-dataset

To dane przygotowane przez Almeida et al. na podstawie forum brytyjskiego, gdzie użytkownicy skarżą się na spamowe SMSy. Każdy wiersz składa się z kolumny opisującej czy wiadomość jest spamem, czy nie ('spam' czy 'ham'), a druga zawiera treść wiadomości.

Proszę pobrać repozytorium ```uczenie-maszynowe-2021-22``` z serwisu github i uaktualnić ścieżkę do danych.

In [ ]:
#!git clone https://github.com/akalinow/uczenie-maszynowe-2021-22
folder = './dane/'

In [ ]:
df = pd.read_csv(...)
print(df)

Dane zawierają zbyteczne kolumny. Proszę:
* usunać kolumny zawierająca wartości "NaN"
* zmienić nazwy kolumn "v1" i "v2" na "label", "text"

**Wskazówka**: proszę użyć metod ```DataFrame.drop()``` oraz ```DataFrame.rename()```

In [ ]:
df = df....
df = df....
df.head()

Proszę wypisać na ekran treść maila o indeksie **57**

**Wskazówka**: Indeksy obiektu DataFreame uzyskujemy przez pole ```DataFrame.index```

In [ ]:
index = 57
print(df[...])

Treść wiadomości:

Proszę wypisać na ekran liczebność danych, czyli liczbę maili.

In [ ]:
print("Data contains {} emails.".format(...))

## Analiza częstości występowania słów w obu klasach za pomocą biblioteki WordCloud

To biblioteka pozwalająca generować śliczne obrazki, na których wielkość słów odpowiada częstości jego występowania w danym zbiorze.

In [ ]:
!pip3 install wordcloud
import wordcloud

In [ ]:
from wordcloud import WordCloud

# najpierw słowa ze spamu
spam_words = " ".join(list(df [df['label']=='spam']['text'] ))
spam_plot = WordCloud(width = 512, height = 512).generate(spam_words)

plt.figure(figsize=(10,8))
plt.imshow(spam_plot);

# teraz słowa z normalnych wiadomosci
ham_words = " ".join(list(df [df['label']=='ham']['text'] ))
ham_plot = WordCloud(width = 512, height = 512).generate(ham_words)

plt.figure(figsize=(10,8))
plt.imshow(ham_plot);

Dane w tej chwili są w postaci ciągów słów. Zamienimy je na postać numeryczną używająć algorymtu ```CountVectorizer```.
Na początek zróbmy to dla prostego tekstu by zrozumieć jak działa ten algorytm. Proszę:
* zaimportować bibliotekę zawierającą algorytm:

```
from sklearn.feature_extraction.text import CountVectorizer
```
* używając metody ```CountVectorizer.fit(...)``` przeprowadzić trening algorytmu na zdaniach 

```
["Ala ma kota.", "Kot? Kot ma wszy.]
```
* wypisać na ekran słownik utworzony przez algorytm ```CountVectorizer```: ```CountVectorizer.vocabulary_```
* wypisać na ekran listę znalezionych słów: ```CountVectorizer.get_feature_names()```
* przepowadzić transformację zdania do postaci numerycznej: ```CountVectorizer.transform(...)```
* przeprowadzić transformację odwrotną:```vectorizer.inverse_transform(...)```
* wypisać na ekran wszystkie reprezentacje zdania

In [ ]:
from ...
text = ...
vectorizer = CountVectorizer()
vectorizer.fit(text)
print("Vocabulary:",...)
print("Lista słów:",...)
text_transformed = vectorizer....
print("Original text:",text)
print("Transformed text:",text_transformed)
print("Transformed text after decoding",...)

Proszę:

* przeprowadzić procedurę treningu i transformacji dla danych z e-maili.
* proszę wypisać na ekran postać oryginalną i po transformacji maila o indeksie **57**.

In [ ]:
vectorizer = ...
vectorizer....
text = ...
text_transformed = ...

print(...)
print(...)
print(...)

## Trening klasyfikatora

Proszę:
* podzielić dane na część uczącą i treningową w stosunku **7:3**
* wytrenować klasyfikator mail korzysjając z naiwnego algorytmu Bayesa opartego o rozkład wielomianowy: ```MultinomialNB```

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(...)

# zaimportuj odpowiednią bibliotekę
from sklearn.naive_bayes import ...
# stwórz obiekt klasyfikatora
model = ...
# naucz klasyfikator na zbiorze uczącym
model....

## Ocena jakości

Korzystając z funkcji napisanych na poprzednich ćwiczeniach:

* wykonać predykcję na danych testowych
* wypisać na ekran wartości metryk o raz macierz pomyłek

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def printScores(model, X, Y):
    ...


printScores(model, X_test, Y_test)

## Analiza modelu

Sprawdźmy, czego właściwie maszyna się nauczyła. Analizując współczynniki modelu proszę wskazać słowa które są istotne dkla klasyfikacji.

Proszę:
* wypisać na ekran współczynniki przypisane do kolejnych słów: ```MultinomialNB.coef_[0]```
* stworzyć listę zawierającą indeksy posortowanych współczynników: ```np.argsort(...)```
* wypisać na ekran po 10 słów o największych i najmniejszych wartościach współczynników.

**Wskazówka** by listę słów moć adresować listą indeksów, listę słów trzeba zamienić na macierz numpy.

In [ ]:
# np.argsort zwraca indeksy w oryginalnej tablicy, które odpowiadają posortowanej tablicy, np.:
# np.argsort([3,1,2]) ---> [1,2,0]
# .coef_[0] zwraca nam tablicę współczynników -- po jednym dla każdej cechy. 
# Wkład do decyzji klasyfikatora jest proporcjonalny do wartości tych współczynników -- większy współczynnik = ważniejsza cecha
feature_names = vectorizer.get_feature_names()
feature_names = np.array(feature_names)
coeff = model.coef_[0]
top10 = np.argsort(coeff)[-10:]
bottom10 = np.argsort(coeff)[:10]

print("Słowa, które z największą pewnością wskazują maszynie, że wiadomość to spam:")
print(feature_names[top10])

print("Słowa najmniej istotne przy klasyfikacji:")
print(feature_names[bottom10])

## Działanie modelu na nieznanych słowach.

Model jedynie rozpoznaje poszczególne słowa, be żadnej analizy językowej. Dodamy pewien element analizy językowej.
Proszę:

* wybrać dowolne, bezsensowne słowo które nie występuje w słowniku i sprawdzić, że tak jest
* dokończyć zdanie : ``Life is`` tym słowem
* wypisać postać ptrzetransformowaną, oraz jej transformację odwrotną
* sprawdzić jak na nie zareaguje model

In [ ]:
word = "...
index = np.where(feature_names == word) 
print("Indeks słowa {} to: {}".format(word, index))

message = ...
message_transformed = ...

print("Original text:",message)
print(...)
print(...)
           
result = ...
print("Model result for senstence: \n {} is {}".format(message,result))

### Stemming (nawet nie będę próbować tego tłumaczyć na polski, to [bogate](https://pl.bab.la/slownik/angielski-polski/stemming) znaczeniowo słowo)

Polega na ujednoliceniu słów o tym samym rdzeniu znaczeniowym (o czym maszyna, oczywiście, nie ma szans wiedzieć). Np. dzięki stemmingowi słowa "go", "going" i "goes" są przyporządkowane tylko jednemu słowu "go". Można np. użyć gotowego algorytmu stemmingowego o nazwie [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/).

Proszę:

* wypisać na ekran indeksy słów ``going`` oraz ``go``

In [ ]:
id1 = np.where(feature_names == 'going')
id2 = ...
print("Indices for words going and go are: "...)

In [ ]:
!pip3 install nltk
import nltk
import ssl
# chcemy pobrac wytrenowany dla języka angielskeigo tokenizer Punkt
# poniższe linijki mają pomóc w przypadku problemów z ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
# pobieramy tokenizer
nltk.download('punkt')
# ładujemy przydatne funkcje
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [ ]:
# przykładowy tekst
message = 'Applying classical methods of machine learning to the study of quantum systems (sometimes called quantum machine learning) is the focus of an emergent area of physics research'
# tokenizujemy czyli dzielimy tekst na słowa
words = word_tokenize(message)

print("Message:\n",message)
print("tokens:\n",words)

# stwórz obiekt typu PorterStemmer
stemmer = PorterStemmer()
# użyj nowo stworzonego obiektu, żeby 'dostać temat' każdego ze słów
words = [stemmer.stem(word) for word in words]
print("words stems:\n",words)

Użycie klasy ```PorterStemmer()``` jako etapu pośdrednie między surowym tekstem, and ```CountVectorizer``` wymaga zdefiniowania obiektu ```analyzer```:

In [ ]:
stemmer = PorterStemmer()
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

vectorizer = CountVectorizer(analyzer=stemmed_words)

Powtórzmy trening i testowanie naszego klasyfikatora na danych poddanych stemmingowi. Proszę:

* stworzyć zmienną zawierająca kolumnę treścni maili
* poddać ją traqnsformacji za użyciem klasy ```PorterStemmer```
* podzielić dane na części uczącą i testową
* następnie wytrenować i poddać transformacji za pomocą ```CountVectorizer```
* następnie wytrenować model i wypisać dla niego wartości metryk

**Wskazówka** do transformacji tekstu użyć obiektu ```vectorizer``` zdefiniowanego w komórce powyżej.

### Wracamy do pracy nad zbiorem mejli

In [ ]:
#przeprowadź serię transformacji
text = ...
text_stemmed = ...
text_transformed = ...

#przeprowadź podział na dane uczące i treningowe
X_train, X_test, Y_train, Y_test = ...

# stwórz obiekt klasyfikatora
model = ...

# naucz klasyfikator na zbiorze uczącym
...

# wypisz wartości metryk
...

## Gdybyście byli spammerami... Co moglibyście zrobić, znając tę technikę antyspamową?

### Stosowanie znaków specjalnych zamiast liter

In [ ]:
our_message = vectorizer.transform(['call for free'])
print(model.predict(our_message))
# podmieniamy literkę a na małpę @
our_tricky_message = vectorizer.transform([...])
print(model.predict(our_tricky_message))

Jakieś inne pomysły? :)

In [ ]:
our_tricky_message = vectorizer.transform(['Call for free sex otherwise you miss a very important meeting'])
print(clf.predict(our_tricky_message))

#### Dlaczego powyższa wiadomość została sklasyfikowana jako pożądana, chociaż jest ewidentnym przykładem spamu?